# find the centroid and radius for a given sequence

In [14]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
cols = plt.cm.get_cmap("rainbow")(
      np.linspace(0, 1.0, 10))[:, :4]

In [18]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

In [21]:
values = np.linspace(0, 1.0, 10)

In [23]:
for val in values:
    print(rgb(0.0, 1.0, val))

(0, 0, 255)
(0, 57, 198)
(0, 114, 141)
(0, 170, 85)
(0, 227, 28)
(28, 227, 0)
(84, 171, 0)
(141, 114, 0)
(198, 57, 0)
(255, 0, 0)


In [2]:
from data.don_data_loader import DonDataLoader
import data.utils as occnet_utils

In [3]:
dataloader = DonDataLoader()

In [5]:
dataloader.get_random_data_pair()

scene: 2018-04-16-15-23-41
frame_idx_a: 1899, frame_idx_b: 1568


(array([[533.6422696 ,   0.        , 319.40910308],
        [  0.        , 534.78244452, 236.43742997],
        [  0.        ,   0.        ,   1.        ]]),
 [<PIL.Image.Image image mode=RGB size=640x480 at 0x7F3A3B76EF28>,
  <PIL.PngImagePlugin.PngImageFile image mode=I size=640x480 at 0x7F3A3B7D10B8>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=640x480 at 0x7F3A3B783F28>,
  array([[ 0.48630148,  0.08533026,  0.86961464,  0.17925081],
         [ 0.5546506 , -0.7991581 , -0.23175213,  0.31838563],
         [ 0.67518411,  0.59503368, -0.43596025,  0.6841579 ],
         [ 0.        ,  0.        ,  0.        ,  1.        ]])],
 [<PIL.Image.Image image mode=RGB size=640x480 at 0x7F3A3B77FF60>,
  <PIL.PngImagePlugin.PngImageFile image mode=I size=640x480 at 0x7F3A3B783940>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=640x480 at 0x7F3A3B783518>,
  array([[ 0.82332536,  0.40973646,  0.3927485 ,  0.48588609],
         [ 0.5207084 , -0.26996621, -0.80992654,  0.52176226],
   

In [7]:
dataloader.centroid_and_radius

{'2018-04-16-15-23-41': {'centroid': 0.8455000000000001,
  'radius': 0.38150000000000006}}

In [9]:
scene_name = "2018-04-16-15-23-41"

In [10]:
all_frames = list(dataloader.dataset.get_pose_data(scene_name).keys())
global_min_depth = float("inf")
global_max_depth = 0.0
for frame in all_frames:
    rgb_a, depth_a, mask_a, pose_a = dataloader.dataset.get_rgbd_mask_pose(scene_name, frame)
    masked_depth = np.array(mask_a)*np.array(depth_a)
    min_depth = masked_depth[masked_depth > 0].min() / 1000.0
    max_depth = masked_depth[masked_depth > 0].max() / 1000.0
    
    global_min_depth = min(global_min_depth, min_depth)
    global_max_depth = max(global_max_depth, max_depth)
z_min = global_min_depth
z_max = global_max_depth

radius = (z_max - z_min) / 2.0
centroid = radius + z_min